# Travelling Salesman

In [ ]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

    
city_names = ["New York", "Los Angeles", "Chicago",
              "Minneapolis", "Denver", "Dallas", "Seattle", "Boston", "San Francisco",
              "St. Louis", "Houston", "Phoenix", "Salt Lake City"]

tsp_size = len(city_names)
# Nodes are indexed from 0 to tsp_size - 1. The depot is the starting node of the route.
depot = 0  
num_routes = 1# The number of routes, which is 1 in the TSP.


routing = pywrapcp.RoutingModel(tsp_size, num_routes, depot)
#search_parameters = routing.DefaultSearchParameters()

#Note: When using GUIDED_LOCAL_SEARCH or other metaheuristics, you need to set a time limit for the solver—otherwise the solver will not terminate. The following code sets a time limit of 30000 milliseconds, or 30 seconds.
search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit_ms = 11000



class CreateDistanceCallback(object):
    """Create callback to calculate distances between points."""
    def __init__(self):
        """Array of distances between points."""
        self.matrix = [
            [   0, 1,  713, 1018, 1631, 1374, 2408,  213, 2571,  875, 1420, 2145, 1972], # New York
            [1,    0, 1, 1524,  831, 1240,  959, 2596,  403, 1589, 1374,  357,  579], # Los Angeles
            [ 713, 1,    0,  1,  920,  803, 1737,  851, 1858,  262,  940, 1453, 1260], # Chicago
            [1018, 1524,  1,    0,  1,  862, 1395, 1123, 1584,  466, 1056, 1280,  987], # Minneapolis
            [1631,  831,  920,  1,    0,  1, 1021, 1769,  949,  796,  879,  586,  371], # Denver
            [1374, 1240,  803,  862,  1,    0, 1, 1551, 1765,  547,  225,  887,  999], # Dallas
            [2408,  959, 1737, 1395, 1021, 1,    0, 1,  678, 1724, 1891, 1114,  701], # Seattle
            [ 213, 2596,  851, 1123, 1769, 1551, 1,    0, 1, 1038, 1605, 2300, 2099], # Boston
            [2571,  403, 1858, 1584,  949, 1765,  678, 1,    0, 1, 1645,  653,  600], # San Francisco
            [ 875, 1589,  262,  466,  796,  547, 1724, 1038, 1,    0,  1, 1272, 1162], # St. Louis
            [1420, 1374,  940, 1056,  879,  225, 1891, 1605, 1645,  1,    0, 1, 1200], # Houston
            [2145,  357, 1453, 1280,  586,  887, 1114, 2300,  653, 1272, 1,    0,  1], # Phoenix
            [1972,  579, 1260,  987,  371,  999,  701, 2099,  600, 1162,  1200,  1,   0]] # Salt Lake City
    def Distance(self, from_node, to_node):
        return int(self.matrix[from_node][to_node])

dist_between_nodes = CreateDistanceCallback()
dist_callback = dist_between_nodes.Distance

routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
assignment = routing.SolveWithParameters(search_parameters)

assignment = routing.SolveWithParameters(search_parameters)



In [ ]:
print("Total distance: " + str(assignment.ObjectiveValue()) + " miles\n")
index = routing.Start(0)
route = []
while not routing.IsEnd(index):
    route.append(city_names[routing.IndexToNode(index)])
    index = assignment.Value(routing.NextVar(index))

print(route)

In [ ]:
['Denver', 'Salt Lake City', 'Seattle', 'San Francisco', 'Los Angeles', 'Phoenix', 'Houston', 'Dallas', 'St. Louis', 'New York', 'Boston', 'Chicago', 'Minneapolis']

### GENERALISATION TO MORE VEHICLES

In [ ]:
import math
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

## DISTANCE ##
def distance(x1,x2,y1,y2):
    dist = abs(x1 - x2) + abs(y1 - y2)
    return(dist)

class CreateDistanceCallbackFunction(object):
    '''returns function to calcualte distance'''
    def __init__(self, locations):
        size = len(locations)
        self.matrix = {}
        
        #make matrix
        for from_node in range(size):
            self.matrix[from_node] = {}
            for to_node in range(size):
                x1 = locations[from_node][0]
                x2 = locations[from_node][1]
                y1 = locations[to_node][0]
                y2 = locations[to_node][1]
                dist = distance(x1,y1,x2,y2)
                self.matrix[from_node][to_node]=dist
    
    def Distance(self, from_node, to_node):
        return(int(self.matrix[from_node][to_node]))

## Demand callback ##

class CreateDemandCallback(object):
    
    def __init__(self,demands):
        self.matrix = demands
    
    def Demand(self, from_node, to_node):
        return(self.matrix[from_node])

def create_data_array():
    locations = [[82, 76], [96, 44], [50, 5], [49, 8], [13, 7], [29, 89], [58, 30], [84, 39],
                 [14, 24], [12, 39], [3, 82], [5, 10], [98, 52], [84, 25], [61, 59], [1, 65],
                 [88, 51], [91, 2], [19, 32], [93, 3], [50, 93], [98, 14], [5, 42], [42, 9],
                 [61, 62], [9, 97], [80, 55], [57, 69], [23, 15], [20, 70], [85, 60], [98, 5]]
    
    demands = [0, 19, 21, 6, 19, 7, 12, 16, 6, 16, 8, 14, 21, 16, 3, 22, 18,
               19, 1, 24, 8, 12, 4, 8, 24, 24, 2, 20, 15, 2, 14, 9]
    
    data = [locations, demands]
    return(data)
    

def main():
    
    data = create_data_array()
    locations = data[0]
    demands = data [1]
    num_locations = len(locations)
    num_vehicles = 5
    
    start_locations = [0, 3, 15, 21, 7]
    end_locations = [2, 20, 7, 11, 1]
    
    routing = pywrapcp.RoutingModel(num_locations, num_vehicles,
                                start_locations, end_locations)
    
    search_parameters = routing.DefaultSearchParameters()
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit_ms = 11000
    
    # setting the arc travel cost
    dist_between_locations = CreateDistanceCallbackFunction(locations)
    dist_callback = dist_between_locations.Distance
    routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
    
    # the demand of each node
    demand_at_locations = CreateDemandCallback(demands)
    demands_callback = demands_at_locations.Demand
    slack_max = 0
    vehicle_capacity = 100
    fix_start_cumul_to_zero = True
    demand = 'Demand'
    routing.AddDimension(demands_callback, slack_max, vehicle_capacity, fix_start_cumul_to_zero, demand)
    
    ### SOLVE ###
    assignment = routing.SolveWithParameters(search_parameters)
    #############
    
    # Report results
    print('Total cost', assignment.ObjectiveValue())
    
    for vehicle_number in range(num_vehicles):
        index = routing.Start(vehicle_number)
        index_next = assignment.Value(routing.NextVar(index))
        route = ''
        route_dist = 0
        route_demand = 0
        
        while not routing.IsEnd(index_next):
            node_index = routing.IndexToNode(index)
            node_index_next = routing.IndexToNode(index_next)
            route += str(node_index) + ' -> '
            route_dist += dist_callback(node_index, node_index_next)
            route_demand += demands[node_index_next]
            
            index = index_next
            index_next = assignment.Value(routing.NextVar(index))
            
        # handle the last portion
        node_index = routing.IndexToNode(index)
        node_index_next = routing.IndexToNode(index_next)
        route += str(node_index) + " -> " + str(node_index_next)
        route_dist += dist_callback(node_index, node_index_next)
        print("Route for vehicle " , str(vehicle_number) , ":\n\n" , route , "\n")
        print("Distance of route " , str(vehicle_number) , ": " , str(route_dist))
        print("Demand met by vehicle " , str(vehicle_number) ,": " , str(route_demand) , "\n")
    
if __name__ == '__main__':
    main()   

# Multiple vehicles and with time window constraints

In [ ]:
import math
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2


## THE DATA ##
def create_data_array():
    locations = [[820, 760], [960, 440], [500, 50], [490, 80], [130, 70], [290, 890], [580, 300],
                 [840, 390], [140, 240], [120, 390], [30, 820], [50, 100], [980, 520], [840, 250],
                 [610, 590], [10, 650], [880, 510], [910, 20], [190, 320], [930, 30], [500, 930],
                 [980, 140], [50, 420], [420, 90], [610, 620], [90, 970], [800, 550], [570, 690],
                 [230, 150], [200, 700], [850, 600], [980, 50]]
    demands =  [0, 19, 21, 6, 19, 7, 12, 16, 6, 16, 8, 14, 21, 16, 3, 22, 18,
                19, 1, 24, 8, 12, 4, 8, 24, 24, 2, 20, 15, 2, 14, 9]
    
    start_times =  [0, 508, 103, 493, 225, 531, 89,
                    565, 540, 108, 602, 466, 356, 303,
                    399, 382, 362, 521, 23, 489, 445,
                    318, 380, 55, 574, 515, 110, 310,
                    387, 491, 328, 73]

    # tw_duration is the width of the time windows.
    tw_duration = 2150

    # In this example, the width is the same at each location, so we define the end times to be
    # start times + tw_duration. For problems in which the time window widths vary by location,
    # you can explicitly define the list of end_times, as we have done for start_times.
    end_times = [0] * len(start_times)

    for i in range(len(start_times)):
        end_times[i] = start_times[i] + tw_duration
    
    data = [locations, demands, start_times, end_times]
    return data


## stuff for costs/callbacks ##
def distance(x1,x2,y1,y2):
    return(abs(x1-x2)+abs(y1-y2))

class CreateDistanceCallback(object):
    def __init__(self, locations):
        self.matrix = {}
        size = len(locations)
        for from_node in range(size):
            self.matrix[from_node] = {}
            for to_node in range(size):
                x1 = locations[from_node][0]
                x2 = locations[from_node][1]
                y1 = locations[to_node][0]
                y2 = locations[to_node][1]
                self.matrix[from_node][to_node] = distance(x1,x2,y1,y2)
    
    def Distance(self, from_node, to_node):
        return(int(self.matrix[from_node][to_node]))
    
    
class CreateDemandCallback(object):
    def __init__(self, demands):
        self.demands = demands
    def Demand(self, from_node, to_node):
        return(self.demands[from_node])
    

class CreateServiceTimeCallback(object):
    def __init__(self, demands, time_per_demand_unit):
        self.demands=demands
        self.time_per_demand_unit = time_per_demand_unit
    def ServiceTime(self, from_node, to_node):
        return(int(self.demands[from_node] * self.time_per_demand_unit))
    
class CreateTravelTimeCallback(object):
    def __init__(self, distance_callback, speed):
        self.dist_callback = distance_callback
        self.speed = speed
    def TravelTime(self, from_node, to_node):
        return(int(self.dist_callback(from_node, to_node) / self.speed))
    

class CreateTotalTimeCallback(object):
    def __init__(self, service_time_callback, travel_time_callback):
        self.service_time_callback = service_time_callback
        self.travel_time_callback = travel_time_callback
    
    def TotalTime(self, from_node, to_node):
        tt = self.service_time_callback(from_node, to_node) + self.travel_time_callback(from_node, to_node)
        return(int(tt))
    
def main():
    data = create_data_array()
    locations = data[0]
    demands = data[1]
    start_times = data[2]
    end_times = data[3]
    num_locations = len(locations)
    depot = 0
    num_vehicles = 5
    search_time_limit = int(4e3)
    vehicle_capacity = 100
    
    routing = pywrapcp.RoutingModel(num_locations, num_vehicles, depot)
    search_parameters = routing.DefaultSearchParameters()

    # Callbacks to the distance function and travel time functions here.
    dist_between_locations = CreateDistanceCallback(locations)
    dist_callback = dist_between_locations.Distance

    routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
    demands_at_locations = CreateDemandCallback(demands)
    demands_callback = demands_at_locations.Demand

    # Adding capacity dimension constraints.
    VehicleCapacity = 100;
    NullCapacitySlack = 0;
    fix_start_cumul_to_zero = True
    capacity = "Capacity"

    routing.AddDimension(demands_callback, NullCapacitySlack, VehicleCapacity,
                         fix_start_cumul_to_zero, capacity)
    # Add time dimension.
    time_per_demand_unit = 3
    horizon = 24 * 3600
    time = "Time"
    speed = 1
    
    #time
    time_per_demand_unit = 3
    horizon = 24 * 3600
    speed = 1
    
    service_times = CreateServiceTimeCallback(demands, time_per_demand_unit)
    service_time_callback = service_times.ServiceTime

    travel_times = CreateTravelTimeCallback(dist_callback, speed)
    travel_time_callback = travel_times.TravelTime

    total_times = CreateTotalTimeCallback(service_time_callback, travel_time_callback)
    total_time_callback = total_times.TotalTime

    routing.AddDimension(total_time_callback,  # total time function callback
                         horizon,
                         horizon,
                         fix_start_cumul_to_zero,
                         time)
    
    ## adding window constraints
    time_dimension = routing.GetDimensionOrDie('time')
    for location in range(1, num_locations):
        start_ = start_times[location]
        end_ = end_times[location]
        time_dimension.CumulVar(location).SetRange(start_, end_)
        
    
    ### SOLVE ###
    print('solvin')
    assignment = routing.SolveWithParameters(search_parameters)
    print('done solvin')
    
    ## display results:
    size = len(locations)
    print('distance of all routes', assignment.ObjectiveValue())
    
    demand_dim = routing.GetDimensionOrDie('demand')
    time_dim = routing.GetDimensionOrDie('time')
    
    #vehics
    for vehicle_number in range(num_vehicles):
        index = routing.Start(vehicle_number)
        plan_output = 'Route {0}:'.format(vehicle_nbr)
        
        while not routing.IsEnd(index):
            node_index = routing.IndexToNode(index)
            load_var = demand_dim.CumulVar(index)
            time_var = time_dim.CumulVar(index)
            plan_output += \
                    " {node_index} Load({load}) Time({tmin}, {tmax}) -> ".format(
                        node_index=node_index,
                        load=assignment.Value(load_var),
                        tmin=str(assignment.Min(time_var)),
                        tmax=str(assignment.Max(time_var)))
            index = assignment.Value(routing.NextVar(index))
        
        node_index = routing.IndexToNode(index)
        load_var = capacity_dimension.CumulVar(index)
        time_var = time_dimension.CumulVar(index)
        plan_output += \
                  " {node_index} Load({load}) Time({tmin}, {tmax})".format(
                      node_index=node_index,
                      load=assignment.Value(load_var),
                      tmin=str(assignment.Min(time_var)),
                      tmax=str(assignment.Max(time_var)))
        print(plan_output)
        print("\n")
        
    return([assignment, routing])

solution = main()

In [1]:
import math
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance(x1, y1, x2, y2):
    # Manhattan distance
    dist = abs(x1 - x2) + abs(y1 - y2)

    return dist

# Distance callback

class CreateDistanceCallback(object):
  """Create callback to calculate distances and travel times between points."""

  def __init__(self, locations):
    """Initialize distance array."""
    num_locations = len(locations)
    self.matrix = {}

    for from_node in range(num_locations):
      self.matrix[from_node] = {}
      for to_node in range(num_locations):
        x1 = locations[from_node][0]
        y1 = locations[from_node][1]
        x2 = locations[to_node][0]
        y2 = locations[to_node][1]
        self.matrix[from_node][to_node] = distance(x1, y1, x2, y2)

  def Distance(self, from_node, to_node):
    return int(self.matrix[from_node][to_node])


# Demand callback
class CreateDemandCallback(object):
  """Create callback to get demands at location node."""

  def __init__(self, demands):
    self.matrix = demands

  def Demand(self, from_node, to_node):
    return self.matrix[from_node]

# Service time (proportional to demand) callback.
class CreateServiceTimeCallback(object):
  """Create callback to get time windows at each location."""

  def __init__(self, demands, time_per_demand_unit):
    self.matrix = demands
    self.time_per_demand_unit = time_per_demand_unit

  def ServiceTime(self, from_node, to_node):
    return int(self.matrix[from_node] * self.time_per_demand_unit)
# Create the travel time callback (equals distance divided by speed).
class CreateTravelTimeCallback(object):
  """Create callback to get travel times between locations."""

  def __init__(self, dist_callback, speed):
    self.dist_callback = dist_callback
    self.speed = speed

  def TravelTime(self, from_node, to_node):
    travel_time = self.dist_callback(from_node, to_node) / self.speed
    return int(travel_time)
# Create total_time callback (equals service time plus travel time).
class CreateTotalTimeCallback(object):
  """Create callback to get total times between locations."""

  def __init__(self, service_time_callback, travel_time_callback):
    self.service_time_callback = service_time_callback
    self.travel_time_callback = travel_time_callback

  def TotalTime(self, from_node, to_node):
    service_time = self.service_time_callback(from_node, to_node)
    travel_time = self.travel_time_callback(from_node, to_node)
    return service_time + travel_time
def main():
  # Create the data.
  data = create_data_array()
  locations = data[0]
  demands = data[1]
  start_times = data[2]
  end_times = data[3]
  num_locations = len(locations)
  depot = 0
  num_vehicles = 5
  search_time_limit = 400000

  # Create routing model.
  if num_locations > 0:

    # The number of nodes of the VRP is num_locations.
    # Nodes are indexed from 0 to num_locations - 1. By default the start of
    # a route is node 0.
    routing = pywrapcp.RoutingModel(num_locations, num_vehicles, depot)
    search_parameters = routing.DefaultSearchParameters()

    # Callbacks to the distance function and travel time functions here.
    dist_between_locations = CreateDistanceCallback(locations)
    dist_callback = dist_between_locations.Distance

    routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
    demands_at_locations = CreateDemandCallback(demands)
    demands_callback = demands_at_locations.Demand

    # Adding capacity dimension constraints.
    VehicleCapacity = 100;
    NullCapacitySlack = 0;
    fix_start_cumul_to_zero = True
    capacity = "Capacity"

    routing.AddDimension(demands_callback, NullCapacitySlack, VehicleCapacity,
                         fix_start_cumul_to_zero, capacity)
    # Add time dimension.
    time_per_demand_unit = 3
    horizon = 24 * 3600
    time = "Time"
    speed = 1

    service_times = CreateServiceTimeCallback(demands, time_per_demand_unit)
    service_time_callback = service_times.ServiceTime

    travel_times = CreateTravelTimeCallback(dist_callback, speed)
    travel_time_callback = travel_times.TravelTime

    total_times = CreateTotalTimeCallback(service_time_callback, travel_time_callback)
    total_time_callback = total_times.TotalTime

    routing.AddDimension(total_time_callback,  # total time function callback
                         horizon,
                         horizon,
                         fix_start_cumul_to_zero,
                         time)
    # Add time window constraints.
    time_dimension = routing.GetDimensionOrDie(time)
    for location in range(1, num_locations):
      start = start_times[location]
      end = end_times[location]
      time_dimension.CumulVar(location).SetRange(start, end)
    # Solve displays a solution if any.
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
      size = len(locations)
      # Solution cost.
      print("Total distance of all routes: " , str(assignment.ObjectiveValue()) , "\n")
      # Inspect solution.
      capacity_dimension = routing.GetDimensionOrDie(capacity);
      time_dimension = routing.GetDimensionOrDie(time);

      for vehicle_nbr in range(num_vehicles):
        index = routing.Start(vehicle_nbr)
        plan_output = 'Route {0}:'.format(vehicle_nbr)

        while not routing.IsEnd(index):
          node_index = routing.IndexToNode(index)
          load_var = capacity_dimension.CumulVar(index)
          time_var = time_dimension.CumulVar(index)
          plan_output += \
                    " {node_index} Load({load}) Time({tmin}, {tmax}) -> ".format(
                        node_index=node_index,
                        load=assignment.Value(load_var),
                        tmin=str(assignment.Min(time_var)),
                        tmax=str(assignment.Max(time_var)))
          index = assignment.Value(routing.NextVar(index))

        node_index = routing.IndexToNode(index)
        load_var = capacity_dimension.CumulVar(index)
        time_var = time_dimension.CumulVar(index)
        plan_output += \
                  " {node_index} Load({load}) Time({tmin}, {tmax})".format(
                      node_index=node_index,
                      load=assignment.Value(load_var),
                      tmin=str(assignment.Min(time_var)),
                      tmax=str(assignment.Max(time_var)))
        print(plan_output)
        print("\n")
    else:
      print('No solution found.')
  else:
    print('Specify an instance greater than 0.')

def create_data_array():

  locations = [[820, 760], [960, 440], [500, 50], [490, 80], [130, 70], [290, 890], [580, 300],
               [840, 390], [140, 240], [120, 390], [30, 820], [50, 100], [980, 520], [840, 250],
               [610, 590], [10, 650], [880, 510], [910, 20], [190, 320], [930, 30], [500, 930],
               [980, 140], [50, 420], [420, 90], [610, 620], [90, 970], [800, 550], [570, 690],
               [230, 150], [200, 700], [850, 600], [980, 50]]

  demands =  [0, 19, 21, 6, 19, 7, 12, 16, 6, 16, 8, 14, 21, 16, 3, 22, 18,
             19, 1, 24, 8, 12, 4, 8, 24, 24, 2, 20, 15, 2, 14, 9]

  start_times =  [0, 508, 103, 493, 225, 531, 89,
                  565, 540, 108, 602, 466, 356, 303,
                  399, 382, 362, 521, 23, 489, 445,
                  318, 380, 55, 574, 515, 110, 310,
                  387, 491, 328, 73]

  # tw_duration is the width of the time windows.
  tw_duration = 2150

  # In this example, the width is the same at each location, so we define the end times to be
  # start times + tw_duration. For problems in which the time window widths vary by location,
  # you can explicitly define the list of end_times, as we have done for start_times.
  end_times = [0] * len(start_times)

  for i in range(len(start_times)):
    end_times[i] = start_times[i] + tw_duration
  data = [locations, demands, start_times, end_times]
  return data
if __name__ == '__main__':
  main()

Total distance of all routes:  10560 

Route 0: 0 Load(0) Time(0, 0) ->  27 Load(0) Time(320, 330) ->  18 Load(20) Time(1130, 1140) ->  8 Load(21) Time(1263, 1273) ->  11 Load(27) Time(1511, 1521) ->  4 Load(41) Time(1663, 1673) ->  28 Load(60) Time(1900, 1910) ->  23 Load(75) Time(2195, 2205) ->  3 Load(83) Time(2299, 2643) ->  0 Load(89) Time(3327, 86400)


Route 1: 0 Load(0) Time(0, 0) ->  21 Load(0) Time(780, 978) ->  31 Load(12) Time(906, 1104) ->  19 Load(21) Time(1003, 1201) ->  17 Load(45) Time(1105, 1303) ->  2 Load(64) Time(1602, 1800) ->  6 Load(85) Time(1995, 2193) ->  14 Load(97) Time(2351, 2549) ->  0 Load(100) Time(2740, 86400)


Route 2: 0 Load(0) Time(0, 0) ->  20 Load(0) Time(490, 684) ->  5 Load(8) Time(764, 958) ->  25 Load(15) Time(1065, 1259) ->  10 Load(39) Time(1347, 1541) ->  15 Load(47) Time(1561, 1755) ->  22 Load(69) Time(1897, 2091) ->  9 Load(73) Time(2009, 2203) ->  29 Load(89) Time(2447, 2641) ->  0 Load(91) Time(3133, 86400)


Route 3: 0 Load(0) Time(0,